In [ ]:
# First step: Create the first population set
def genesis(city_list, n_population):
    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)),n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

In [ ]:
#individual solution
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
    total += compute_city_distance_names(a,b, cities_dict)
return total

#All solutions
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)
    #Looping over all solutions computing the fitness for each solution
    for i in range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i],cities_dict)
    return fitnes_list

In [ ]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))),len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))),len(population_set),p=prob_list, replace=True)

    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]

    return np.array([progenitor_list_a,progenitor_list_b])

In [ ]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]
    for city in prog_b:
        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))
    return offspring

# Finding pairs of mates
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i],
        progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
    return new_population_set

In [ ]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)
        offspring[a], offspring[b] = offspring[b], offspring[a]
    return offspring

# New populaiton generation
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

In [ ]:
best_solution = [-1,np.inf,np.array([])]

for i in range(100000):

    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(),
        datetime.now().strftime("%d/%m/%y %H:%M"))
        fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
        progenitor_list = progenitor_selection(population_set,fitnes_list)
        new_population_set = mate_population(progenitor_list)
        mutated_pop = mutate_population(new_population_set)

In [1]:
def main():
    cards = [1,2,3,4,5,6,7,8,9,10]
    
    
if __name__=='__main__':
    main()

holleo


In [2]:
import math
import random


# Graph
graph = {
    "a" : [["b", 12], ["c", 10], ["g", 12]],
    "b" : [["a", 12], ["c", 8], ["d", 12]],
    "c" : [["a", 10], ["b", 8], ["d", 11], ["e", 3], ["g",9]],
    "d" : [["b", 12], ["c", 11], ["e", 11], ["f", 10]],
    "e" : [["c", 3], ["d", 11], ["f", 6], ["g", 7]],
    "f" : [["d", 10], ["e", 6], ["g",9]],
    "g" : [["a", 12], ["c", 9], ["e", 7], ["f", 9]]
}

# chromosome
chromosome_size = 8
initial_pop_size = 4
start_node = "a"
crossover_bit_range = [2, 6]

# create initial populaton
def initialise_pop():
    initial_pop = ["abcdefga", "agcefdba", "acbdfega", "agefdcba"]

    for i in initial_pop:
        if (len(i) != chromosome_size) or (i[0] != i[-1] != start_node):
            print("Chromosome", initial_pop.index(i), "is not a Solution!")
            return "Error"

    for j in ["a", "b", "c", "d", "e", "f", "g"]:
        if j not in i:
            print("Chromosome", initial_pop.index(i), "is not a Solution!")
            return "Error"

    return initial_pop


# Cost function
def calc_path_cost(path):
    path_cost = 0
    for i in range(len(path)-1):
        temp = graph.get(path[i])
        if temp is not None:
            for j in temp:
                if j[0] == path[i+1]:
                    path_cost += j[1]
        else:
            path_cost += math.inf
    return path_cost

def calc_fitness(initial_pop):
    fitness = []
    for i in initial_pop:
        fx = calc_path_cost(i)
        fitness_value = 1/(1+fx)
        fitness.append(fitness_value)

    return fitness

# Selection - Roulette's Wheel
def selection(initial_pop, fitness):
    selection_prob, selected_pop = [], []
    sum_fitness = sum(fitness)

    for i in range(len(initial_pop)):
        selection_prob.append(fitness[i]/sum_fitness)

    for i in range(len(initial_pop)):
        temp = random.choices(initial_pop, selection_prob)[0]
        selected_pop.append([temp, fitness[initial_pop.index(temp)]])

    selected_pop.sort(key = lambda x: x[1])
    selected_pop = selected_pop[::-1]
    return selected_pop


# Crossover - ORDER 1
def crossover(new_pop):
    mating_pairs = []

    for i in range(0, len(new_pop), 2):
        par1 = new_pop[i]
        par2 = new_pop[i+1]
        mating_pairs.append([par1, par2])

    start, end = crossover_bit_range[0], crossover_bit_range[1]

    # p1 -> 9 2 3 1 5 6 7 8 4       c1 -> * * * 4 5 6 7 * *     [9]
    # p2 -> 9 3 7 8 2 6 5 1 4       c2 -> 9 * * 8 2 6 5 * 4     [3 1 7] -> 9 1 7 8 2 6 5 3 4

    children = []
    for i in mating_pairs:
        p1, p2 = i[0], i[1]
        child1 = child2 = start_node + "*" * (len(par1)-2) + start_node
        child1 = child1[ : start] + p1[start:end] + child1[end:]
        child2 = child2[ : start] + p2[start:end] + child2[end:]

        seq1, seq2 = [], []
        for j in range(end, len(p2)):
            if p2[j] not in child1:
                seq1.append(p2[j])
            if p1[j] not in child2:
                seq2.append(p1[j])

        for j1 in range(0, end):
            if p2[j1] not in child1:
                seq1.append(p2[j1])
            if p1[j1] not in child2:
                seq2.append(p1[j1])

        for k in range(end, len(child1)):
            if child1[k] == "*":
                child1 = child1[:k] + seq1.pop(0) + child1[k+1:]
            if child2[k] == "*":
                child2 = child2[:k] + seq2.pop(0) + child2[k+1:]

        for l in range(0, start):
            if child1[l] == "*":
                child1 = child1[:l] + seq1.pop(0) + child1[l+1:]
            if child2[l] == "*":
                child2 = child2[:l] + seq2.pop(0) + child2[l+1:]

        children.append([child1, child2])

    return [mating_pairs, children]


# main
def main():
    initial_pop = initialise_pop()
    print("Initial Population: ", initial_pop)
    
    fitness = calc_fitness(initial_pop)
    print("Fitness: ", fitness)
    
    selected_pop = selection(initial_pop, fitness)
    print("Population after selection: ", selected_pop)
    
    new_pop = [x[0] for x in selected_pop]
    
    mc = crossover(new_pop)
    mating_pairs = mc[0]
    child_pairs = mc[1]
    
    for i in range(len(mating_pairs)):
        print("Mating Pairs: ", mating_pairs[i])
        print("Population After Crossover: ", child_pairs[i], "\n")
        
    children_pop = []
    for i in child_pairs:
        children_pop.append(i[0])
        children_pop.append(i[1])
        
    # Evaluate Children Population


if __name__ == "__main__":
    main()

Initial Population:  ['abcdefga', 'agcefdba', 'acbdfega', 'agefdcba']
Fitness:  [0.014285714285714285, 0.015384615384615385, 0.015151515151515152, 0.014925373134328358]
Population after selection:  [['agefdcba', 0.014925373134328358], ['abcdefga', 0.014285714285714285], ['abcdefga', 0.014285714285714285], ['abcdefga', 0.014285714285714285]]
Mating Pairs:  ['agefdcba', 'abcdefga']
Population After Crossover:  ['abefdcga', 'agcdefba'] 

Mating Pairs:  ['abcdefga', 'abcdefga']
Population After Crossover:  ['abcdefga', 'abcdefga'] 

